# Generador de datos sintéticos para actualizaciones de la DB (deltas)

##Instalación de librerías necesarias

In [ ]:
%pip install sdv
%pip install sqlalchemy
%pip install --upgrade pandas
%pip install boto3

Python interpreter will be restarted.
Collecting sdv
 Downloading sdv-0.16.0-py2.py3-none-any.whl (102 kB)
Collecting tqdm<5,>=4.15
 Downloading tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
Collecting graphviz<1,>=0.13.2
 Downloading graphviz-0.20.1-py3-none-any.whl (47 kB)
Collecting ctgan<0.6,>=0.5.1
 Downloading ctgan-0.5.1-py2.py3-none-any.whl (24 kB)
Collecting rdt<0.7,>=0.6.2
 Downloading rdt-0.6.4-py2.py3-none-any.whl (45 kB)
Collecting copulas<0.8,>=0.7.0
 Downloading copulas-0.7.0-py2.py3-none-any.whl (53 kB)
Collecting sdmetrics<0.6,>=0.5.0
 Downloading sdmetrics-0.5.0-py2.py3-none-any.whl (98 kB)
Collecting deepecho<0.4,>=0.3.0.post1
 Downloading deepecho-0.3.0.post1-py2.py3-none-any.whl (26 kB)
Requirement already satisfied: numpy<2,>=1.20.0 in /databricks/python3/lib/python3.8/site-packages (from sdv) (1.20.1)
Requirement already satisfied: pandas<2,>=1.1.3 in /databricks/python3/lib/python3.8/site-packages (from sdv) (1.2.4)
Collecting Faker<10,>=3.0.0
 Downloading Faker-9.9.1-py3-none-any.whl (1.2 MB)
Requirement already satisfied: scipy<2,>=1.5.4 in /databricks/python3/lib/python3.8/site-packages (from copulas<0.8,>=0.7.0->sdv) (1.6.2)
Requirement already satisfied: matplotlib<4,>=3.4.0 in /databricks/python3/lib/python3.8/site-packages (from copulas<0.8,>=0.7.0->sdv) (3.4.2)
Requirement already satisfied: packaging<22,>=20 in /databricks/python3/lib/python3.8/site-packages (from ctgan<0.6,>=0.5.1->sdv) (20.9)
Collecting torch<2,>=1.8.0
 Downloading torch-1.12.0-cp38-cp38-manylinux1_x86_64.whl (776.3 MB)
Collecting torchvision<1,>=0.9.0
 Downloading torchvision-0.13.0-cp38-cp38-manylinux1_x86_64.whl (19.1 MB)
Requirement already satisfied: scikit-learn<2,>=0.24 in /databricks/python3/lib/python3.8/site-packages (from ctgan<0.6,>=0.5.1->sdv) (0.24.1)
Requirement already satisfied: python-dateutil>=2.4 in /databricks/python3/lib/python3.8/site-packages (from Faker<10,>=3.0.0->sdv) (2.8.1)
Collecting text-unidecode==1.3
 Downloading text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<4,>=3.4.0->copulas<0.8,>=0.7.0->sdv) (1.3.1)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<4,>=3.4.0->copulas<0.8,>=0.7.0->sdv) (0.10.0)
Requirement already satisfied: pillow>=6.2.0 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<4,>=3.4.0->copulas<0.8,>=0.7.0->sdv) (8.2.0)
Requirement already satisfied: pyparsing>=2.2.1 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<4,>=3.4.0->copulas<0.8,>=0.7.0->sdv) (2.4.7)
Requirement already satisfied: six in /databricks/python3/lib/python3.8/site-packages (from cycler>=0.10->matplotlib<4,>=3.4.0->copulas<0.8,>=0.7.0->sdv) (1.15.0)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas<2,>=1.1.3->sdv) (2020.5)
Collecting pyyaml<6,>=5.4.1
 Downloading PyYAML-5.4.1-cp38-cp38-manylinux1_x86_64.whl (662 kB)
Collecting psutil<6,>=5.7
 Downloading psutil-5.9.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (284 kB)
Requirement already satisfied: joblib>=0.11 in /databricks/python3/lib/python3.8/site-packages (from scikit-learn<2,>=0.24->ctgan<0.6,>=0.5.1->sdv) (1.0.1)
Requirement already satisfied: threadpoolctl>=2.0.0 in /databricks/python3/lib/python3.8/site-packages (from scikit-learn<2,>=0.24->ctgan<0.6,>=0.5.1->sdv) (2.1.0)
Collecting pyts<0.13.0,>=0.12.0
 Downloading pyts-0.12.0-py3-none-any.whl (2.5 MB)
Collecting numba>=0.48.0
 Downloading numba-0.56.0-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.5 MB)
Requirement already satisfied: setuptools in /usr/local/lib/python3.8/dist-packages (from numba>=0.48.0->pyts<0.13.0,>=0.12.0->sdmetrics<0.6,>=0.5.0->sdv) (52.0.0)
Collecting llvmlite<0.40,>=0.39.0dev0
 Downloading llvmlite-0.39.0-cp38-cp38-manylinux_2_17_x86_64.ma

## Importación de librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import requests
from sqlalchemy import create_engine
from sdv.tabular import GaussianCopula
import random
import boto3
import zipfile
import os
import zlib

## Descarga del modelo ya entrenado

In [ ]:
r = requests.get('https://proyecto-grupal-henry.s3.amazonaws.com/generador_deltas.pkl')
with open('./generador_deltas.pkl', 'wb') as f:
    f.write(r.content)

## Generación de datos sintéticos con el modelo

In [ ]:
model = GaussianCopula()
loaded = model.load('generador_deltas.pkl')
deltas = loaded.sample(num_rows=100000)

--------------------------------------------------------------------------- 
 UnpicklingError Traceback (most recent call last)
 <command-3975064024120600> in <module> 
 1 model = GaussianCopula ( ) 
 ----> 2 loaded = model . load ( 'generador_deltas.pkl' ) 
 3 deltas = loaded . sample ( num_rows = 100000 ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-9b5a5856-b645-4c37-90c7-088710d1b97c/lib/python3.8/site-packages/sdv/tabular/base.py in load (cls, path) 
 896 """
 897 with open ( path , 'rb' ) as f : 
 --> 898 model = pickle . load ( f ) 
 899 throw_version_mismatch_warning ( getattr ( model , '_package_versions' , None ) ) 
 900 

 UnpicklingError : invalid load key, '<'.

## Preparación de los datos sintéticos para que sean coherentes con las fechas y no haya órdenes duplicadas

In [ ]:
# Minimiza el riesgo de duplicados haciendo cambios pseudoaleatorios a las order_id.
def random_id_replacer(string):
    return random.choice(random.choice('0123456789abcdefghijklmnopqrstuvwxyz'))

deltas['order_id'].str.replace('[a-z]', rand, regex=True)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2473785141737992> in <module> 
 1 pachin = '' . join ( random . choice ( '0123456789abcdefghijklmnopqrstuvwxyz' ) for i in range ( 32 ) ) 
 2 
 ----> 3 deltas [ 'order_id' ] = deltas . order_id . apply ( pachin ) 

 NameError : name 'deltas' is not defined

In [ ]:
caracteres=['a','b','c','d','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','1','2','3','4','5','6','7','8','9','0']

pachin = ''.join(random.choice('0123456789abcdefghijklmnopqrstuvwxyz') for i in range(32))

deltas['order_id'] = deltas.order_id.apply(pachin)
deltas.order_id.str.replace(*order_shuffler2)
deltas.order_id.str.replace(*order_shuffler3)
deltas.drop_duplicates('order_id', keep='first', inplace=True)
cond1 = (deltas.order_purchase_timestamp < deltas.order_approved_at)
cond2 = (deltas.order_purchase_timestamp < deltas.order_delivered_carrier_date)
cond3 = (deltas.order_purchase_timestamp < deltas.order_estimated_delivery_date)
cond4 = (deltas.order_approved_at < deltas.order_delivered_carrier_date)
cond5 = (deltas.order_approved_at < deltas.order_estimated_delivery_date)
cond6 = (deltas.order_delivered_customer_date >= deltas.order_delivered_carrier_date)
coherentes = cond1 & cond2 & cond3 & cond4 & cond5 & cond6
delta = deltas[coherentes]

## Generación de las tablas deltas finales desde el delta unificado

In [ ]:
delta_orders = delta[['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', "order_delivered_carrier_date","order_delivered_customer_date","order_estimated_delivery_date"]]
delta_order_items = delta[["order_id","order_item_id","product_id","seller_id","shipping_limit_date","price","freight_value"]]
delta_order_payments = delta[["order_id","payment_sequential","payment_type","payment_installments","payment_value"]]
delta_order_reviews = delta[["review_id","order_id","review_score","review_comment_title","review_comment_message","review_creation_date","review_answer_timestamp"]]
delta_products = delta[["product_id","product_category_name","product_name_lenght","product_description_lenght","product_photos_qty","product_weight_g","product_length_cm","product_height_cm","product_width_cm"]]
delta_sellers = delta[["seller_id","seller_zip_code_prefix","seller_city","seller_state"]]
delta_customers = delta[["customer_id","customer_unique_id","customer_zip_code_prefix","customer_city","customer_state"]]
delta_customers = delta_customers.rename(columns={'customer_zip_code_prefix':'zip_code_prefix', 'customer_city':'city', 'customer_state':'state'})
delta_order_payments = delta_order_payments.drop(columns='payment_sequential')
delta_order_items = delta_order_items.rename(columns={'order_item_id':'product_quantity'})
delta_order_reviews = delta_order_reviews.rename(columns={'review_comment_message':'comment_message', 'review_creation_date':'creation_date', 'review_comment_title':'comment_title', 'review_answer_timestamp':'answer_timestamp'})
delta_orders = delta_orders.rename(columns={'order_purchase_timestamp':'purchase_timestamp', 'order_delivered_carrier_date':'delivered_carrier_date', 'order_delivered_customer_date':'delivered_customer_date', 'order_estimated_delivery_date':'estimated_delivery_date'})
delta_products = delta_products.rename(columns={'product_description_lenght':'description_length', 'product_name_lenght':'name_length', 'product_weight_g':'weight_g', 'product_length_cm':'length_cm', 'product_height_cm':'height_cm', 'product_width_cm':'width_cm', 'product_photos_qty':'photos_quantity'})
delta_sellers = delta_sellers.rename(columns={'seller_zip_code_prefix':'zip_code_prefix', 'seller_city':'city', 'seller_state':'state'})
delta_order_reviews['comment_message'].fillna('sem_comentarios',inplace=True)
delta_order_reviews['comment_title'].fillna('sem_titulo',inplace=True)

## Guardado de los deltas en archivos .csv y luego todos juntos en un archivo .zip

In [ ]:
delta_orders.to_csv('delta_orders.csv')
delta_products.to_csv('delta_products.csv')
delta_sellers.to_csv('delta_sellers.csv')
delta_order_reviews.to_csv('delta_order_reviews.csv')
delta_order_payments.to_csv('delta_order_payments.csv')
delta_customers.to_csv('delta_customers.csv')
delta_order_items.to_csv('delta_order_items.csv')

lista_csvs = ['delta_orders.csv', 'delta_products.csv', 'delta_sellers.csv', 'delta_order_reviews.csv', 'delta_order_payments.csv', 'delta_customers.csv', 'delta_order_items.csv']

with zipfile.ZipFile('deltas.zip', 'w') as zipF:
    for csv in lista_csvs:
        zipF.write(csv, compress_type=zipfile.ZIP_DEFLATED)

In [ ]:
print(os.stat('deltas.zip'))

os.stat_result(st_mode=33188, st_ino=8259846, st_dev=53, st_nlink=1, st_uid=0, st_gid=0, st_size=407679, st_atime=1658380605, st_mtime=1658380870, st_ctime=1658380870)

## Guardado del archivo comprimido con los deltas en el DataLake

In [ ]:
session = boto3.Session(
aws_access_key_id=<access_key>,
aws_secret_access_key=<secret_access_key>
)

#Creating S3 Resource From the Session.
s3 = session.resource('s3')

s3.Bucket('proyecto-grupal-henry').upload_file("deltas.zip", "deltas.zip")

File Uploaded Successfully